In [1]:
import torch
import torch.nn.functional as F
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm import tqdm
import gc
import seaborn as sns
import pickle

import numpy as np
import pandas as pd

from imblearn.under_sampling import RandomUnderSampler 
from sklearn.metrics import classification_report

from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict

from xgboost import XGBRFRegressor, XGBRFClassifier
import xgboost

sns.set_context("notebook", font_scale=1.25)

/Users/bono/miniforge3/envs/ag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('preprocessed.pickle', 'rb') as handle:
    mydict = pickle.load(handle)
    
truth = mydict['truth']
segments = mydict['segments']
m = mydict['m']

In [3]:
mm = m.copy()

### classifier

In [4]:
mm.run.max()

9

In [5]:
mm.segment.unique()

array(['preamble', 'instruction', 'input', 'question', 'postilla',
       'generated'], dtype=object)

In [6]:
# minimum number of generated tokens
mm[(mm.segment=='postilla')].groupby('pid').tokzero.max().min()

64

In [12]:
# beginning of generation
ds = mm[(mm.segment=='postilla')][['run', 'pid', 'tokzero', 'shape', 'true_ent', 'maxp']]

ds = ds.pivot(index=['run', 'pid'], columns='tokzero', values=['maxp', 'true_ent']).reset_index()
ds.columns = [''.join([str(c) for c in col]).strip() for col in ds.columns.values]
ds = ds.merge(truth, on=['run', 'pid'])


In [13]:
ds.correct.value_counts()

correct
True     4977
False    1903
Name: count, dtype: int64

In [14]:
y = ds.correct
pids = ds.pid
X = ds.drop(columns=['run', 'pid', 'correct', 'hallucinated'])

rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
pids_res = pids[y_res.index]

X_res.columns = ['postilla_'+col for col in X.columns]

### beginning of generation

In [16]:
pids_res

0          9
1         10
2         12
3         14
7         23
        ... 
2707    1677
1194    1342
1037     896
2272     530
134      355
Name: pid, Length: 3806, dtype: int64

In [17]:
from sklearn.model_selection import GroupKFold

group_kfold = GroupKFold(n_splits=5)

# just get the first group
for i, (train_index, test_index) in enumerate(group_kfold.split(X_res, y_res, pids_res)):
    print(f"Fold {i}:")
    # print(f"  Train: index={train_index}, group={pids_res.iloc[train_index]}")
    # print(f"  Test:  index={test_index}, group={pids_res.iloc[test_index]}")
    break

Fold 0:


In [18]:
X_train = X_res.iloc[train_index]
y_train = y_res.iloc[train_index]
pids_train = pids_res.iloc[train_index]

X_test = X_res.iloc[test_index]
y_test = y_res.iloc[test_index]
pids_test = pids_res.iloc[test_index]

In [19]:
# must be true
len(set(pids_train.values) & set(pids_test.values))==0

True

In [20]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [21]:
X_train['label'] = y_train
predictor = TabularPredictor(label='label', path='tablellama-high', log_to_file=True).fit(X_train, presets='high_quality')



/var/folders/kn/n2krf1q970d6cfl43cjq73s80000gn/T/ipykernel_74766/1796950693.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['label'] = y_train
/var/folders/kn/n2krf1q970d6cfl43cjq73s80000gn/T/ipykernel_74766/1796950693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['label'] = y_train
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwi

(_dystack pid=74806) 	0.8329	 = Validation score   (accuracy)
(_dystack pid=74806) 	0.01s	 = Training   runtime
(_dystack pid=74806) 	0.04s	 = Validation runtime
(_dystack pid=74806) Fitting model: KNeighborsDist_BAG_L1 ... Training model for up to 597.23s of the 896.55s of remaining time.
(_dystack pid=74806) 	0.8721	 = Validation score   (accuracy)
(_dystack pid=74806) 	0.01s	 = Training   runtime
(_dystack pid=74806) 	0.02s	 = Validation runtime
(_dystack pid=74806) Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 597.19s of the 896.51s of remaining time.
(_dystack pid=74806) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.34%)
(_dystack pid=74806) 	0.8736	 = Validation score   (accuracy)
(_dystack pid=74806) 	2.99s	 = Training   runtime
(_dystack pid=74806) 	0.07s	 = Validation runtime
(_dystack pid=74806) Fitting model: LightGBM_BAG_L1 ... Training model for up to 592.32s of the 891.64s of

(_ray_fit pid=74981) [1000]	valid_set's binary_error: 0.136095


(_dystack pid=74806) 	0.8662	 = Validation score   (accuracy)
(_dystack pid=74806) 	1.78s	 = Training   runtime
(_dystack pid=74806) 	0.12s	 = Validation runtime
(_dystack pid=74806) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 291.16s of the 590.48s of remaining time.
(_dystack pid=74806) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.14%)
(_dystack pid=74806) 	0.8784	 = Validation score   (accuracy)
(_dystack pid=74806) 	18.53s	 = Training   runtime
(_dystack pid=74806) 	0.19s	 = Validation runtime
(_dystack pid=74806) Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 270.87s of the 570.19s of remaining time.
(_dystack pid=74806) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=3.24%)
(_dystack pid=74806) 	0.8717	 = Validation score   (accuracy)
(_dystack pid=74806) 	12.49s	 = Training   run

(_dystack pid=74806) 	0.8739	 = Validation score   (accuracy)
(_dystack pid=74806) 	13.69s	 = Training   runtime
(_dystack pid=74806) 	0.02s	 = Validation runtime
(_dystack pid=74806) Fitting model: NeuralNetFastAI_r103_BAG_L1 ... Training model for up to 68.54s of the 367.86s of remaining time.
(_dystack pid=74806) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.23%)
(_ray_fit pid=75150) No improvement since epoch 11: early stopping
(_dystack pid=74806) 	0.8732	 = Validation score   (accuracy)
(_dystack pid=74806) 	6.8s	 = Training   runtime
(_dystack pid=74806) 	0.07s	 = Validation runtime
(_dystack pid=74806) Fitting model: NeuralNetTorch_r14_BAG_L1 ... Training model for up to 59.97s of the 359.29s of remaining time.
(_dystack pid=74806) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.13%)
(_dystack pid=74806) 	0.8773	 = Val

...

	0.867	 = Validation score   (accuracy)
	1.33s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetTorch_r143_BAG_L1 ... Training model for up to 1822.45s of the 1822.44s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.13%)
	0.8807	 = Validation score   (accuracy)
	16.66s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: CatBoost_r128_BAG_L1 ... Training model for up to 1804.67s of the 1804.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.46%)
	0.8742	 = Validation score   (accuracy)
	114.32s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI_r111_BAG_L1 ... Training model for up to 1688.99s of the 1688.98s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 worker

	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.38%)
	0.8696	 = Validation score   (accuracy)
	5.12s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI_r172_BAG_L1 ... Training model for up to 1520.78s of the 1520.77s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.24%)
	0.8748	 = Validation score   (accuracy)
	3.38s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost_r180_BAG_L1 ... Training model for up to 1516.38s of the 1516.38s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.89%)
	0.8748	 = Validation score   (accuracy)
	58.47s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch_r76_BAG_L1 ... Training model for up to 1456.55s

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting model: KNeighborsUnif_BAG_L1_FULL | Skipping fit via cloning parent ...
	0.01s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: KNeighborsDist_BAG_L1_FULL | Skipping fit via cloning parent ...
	0.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMXT_BAG_L1_FULL ...
	1.66s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_BAG_L1_FULL ...
	1.83s	 = Training   runtime
Fitting model: RandomForestGini_BAG_L1_FULL | Skipping fit via c

Fitting model: RandomForest_r127_BAG_L1_FULL | Skipping fit via cloning parent ...
	2.84s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r134_BAG_L1_FULL ...
No improvement since epoch 0: early stopping
	1.03s	 = Training   runtime
Fitting model: RandomForest_r34_BAG_L1_FULL | Skipping fit via cloning parent ...
	1.61s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r94_BAG_L1_FULL ...
	1.29s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r143_BAG_L1_FULL ...
	28.06s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoost_r128_BAG_L1_FULL ...
	9.44s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r111_BAG_L1_FULL ...
	Stopping at the best epoch learned earlier - 18.
	0.6s	 =

TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/Users/bono/Library/CloudStorage/OneDrive-PolitecnicodiMilano/work/prin/llm-uncertainty/tablellama-high")


In [22]:
# X_train['label'] = y_train
# predictions = predictor.predict(X_test)

In [23]:
pred_proba = predictor.predict_proba(X_test)
pred = pred_proba.values.argmax(axis=1).astype(bool)
print(classification_report(y_test.values, pred))

              precision    recall  f1-score   support

       False       0.58      0.40      0.47       371
        True       0.56      0.73      0.63       391

    accuracy                           0.57       762
   macro avg       0.57      0.56      0.55       762
weighted avg       0.57      0.57      0.55       762



In [24]:
%%time

from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict

from xgboost import XGBRFRegressor, XGBRFClassifier
import xgboost

#partition cv by prompt
xgb_model = XGBRFClassifier(n_estimators=100, n_jobs=6)

# pred = cross_val_predict(xgb_model, X, y, cv=GroupKFold(n_splits=10), groups=pids)
# pred_proba = cross_val_predict(xgb_model, X, y, cv=LeaveOneGroupOut(), groups=pids, method='predict_proba')

# pred_proba = cross_val_predict(xgb_model, X_res, y_res, cv=GroupKFold(n_splits=10), groups=pids_res, method='predict_proba')

xgb_model.fit(X_train.drop(columns=['label']), y_train)
pred_proba = xgb_model.predict_proba(X_test)

CPU times: user 1.34 s, sys: 204 ms, total: 1.55 s
Wall time: 543 ms


In [25]:
pred = pred_proba.argmax(axis=1).astype(bool)
print(classification_report(y_test.values, pred))

              precision    recall  f1-score   support

       False       0.65      0.50      0.56       371
        True       0.61      0.74      0.67       391

    accuracy                           0.62       762
   macro avg       0.63      0.62      0.62       762
weighted avg       0.63      0.62      0.62       762



In [26]:
%%time

from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict

from xgboost import XGBRFRegressor, XGBRFClassifier
import xgboost

#partition cv by prompt
xgb_model = XGBRFClassifier(n_estimators=100, n_jobs=6)

# pred = cross_val_predict(xgb_model, X, y, cv=GroupKFold(n_splits=10), groups=pids)
# pred_proba = cross_val_predict(xgb_model, X, y, cv=LeaveOneGroupOut(), groups=pids, method='predict_proba')

pred_proba = cross_val_predict(xgb_model, X_res, y_res, cv=GroupKFold(n_splits=10), groups=pids_res, method='predict_proba')


CPU times: user 14.1 s, sys: 1.57 s, total: 15.6 s
Wall time: 5.13 s


In [ ]:
pred = pred_proba.argmax(axis=1).astype(bool)
print(classification_report(y_test.values, pred))